In [ ]:
from h5rdmtoolbox.h5wrapper import H5Flow

# H5Flow

The wrapper class `H5Flow` is a specialized group inherite from `H5File`. It adds domain-specific functionality for working with fluid data. It is still implemented as a general purpose HDF5 interface but specifically to fluid data.

As we deal with fluid data, we expect to have velocities in the file. Let's generate an HDF5 file with random data. Note, that `create_coordinates()` and `create_velocity_datasets()` are new methods as compared to `H5File`:

In [ ]:
import numpy as np
with H5Flow() as h5:
    h5.create_coordinates(x=np.linspace(0, 1, 20),
                          y=np.linspace(0, 0.5, 10),
                          z=np.linspace(-1, 1, 3),
                          coords_unit='mm')
    h5.create_velocity_datasets(u=np.random.rand(3, 10, 20),
                                v=np.random.rand(3, 10, 20),
                                w=np.random.rand(3, 10, 20),
                                dim_scales=('z', 'y', 'x'),
                                units='mm/s')
    h5.dump()
    filename = h5.hdf_filename

## Vector data access

The `H5Flow` class has "special dataset" accessors like `Vector` and `VelocityVector`. The latter one without a call will try to find the dataset based on the `standard_name` attribute (TODO: make it dependend on the convention as it is different f cf and CGNS).

In [ ]:
with H5Flow(filename) as h5:
    vel = h5.Vector(names=('u', 'v', 'w'))[0:1, :, :]

In [ ]:
vel

In [ ]:
with H5Flow(filename) as h5:
    vel = h5.VelocityVector[0:1, :, :]
vel

## Devices
Assume we have experiment data, e.g. we measured the pressure. We can reference the measurement device in the dataset like so:

In [ ]:
with H5Flow() as h5:
    h5.create_group('devices')
    device_grp = h5.create_group('devices/PressureSensor')
    device_grp.attrs['manufacturer'] = 'unknown'
    device_grp.create_dataset('x', data=0, units='m', standard_name='x_coordinate')
    device_grp.create_dataset('y', data=0, units='m', standard_name='y_coordinate')
    device_grp.create_dataset('z', data=0, units='m', standard_name='z_coordinate')
    ds = h5.create_dataset('pressure', data=np.random.random(100), units='Pa', standard_name='pressure',
                          device=device_grp)
    print(ds.device)
    print(type(ds.device))

**Alternative 1:** We can first use the `Device` class to create an object, which we write to the hdf group and then create the reference:

In [ ]:
from h5rdmtoolbox.h5wrapper.h5flow import Device
p_sensor = Device('PressureSensor', manufacturer='unknown',
                  x=(0, dict(units='m',standard_name='x_coordinate')),
                  y=(0, dict(units='m',standard_name='y_coordinate')),
                  z=(0, dict(units='m',standard_name='z_coordinate')))
p_sensor

In [ ]:
with H5Flow() as h5:
    devices_grp = h5.create_group('devices')
    sensor_grp = p_sensor.to_hdf_group(devices_grp)
    
    ds = h5.create_dataset('pressure', data=np.random.random(100), units='Pa', standard_name='pressure',
                          device=sensor_grp)
    ds.device = sensor_grp
    print(ds.device)
    print(type(ds.device))

In [ ]:
with H5Flow() as h5:    
    ds = h5.create_dataset('pressure', data=np.random.random(100), units='Pa', standard_name='pressure')
    ds.device = p_sensor

# Create two flow types (Poiseuille and rankine vortex)

In [ ]:
from h5rdmtoolbox import tutorial

In [ ]:
vel_couette = tutorial.get_xr_dataset('couette2d')

The above variables are of type `xarray.Dataset`:

In [ ]:
vel_couette

The velocity component can directly be plotted with coorect units at the labels:

In [ ]:
vel_couette.u.plot()

# Add both flow types to individual groups

A `xarray.Dataset` can be stored in an HDF file as done in the following:

In [ ]:
vel_poiseuille = tutorial.get_xr_dataset('poiseuille2D')
vel_poiseuille

In [ ]:
with H5Flow(mode='w') as h5:
    h5.create_dataset('vel', data=[1,2,3], standard_name='x_velocity', units='m/s')
    print(type(h5.vel.standard_name))

In [ ]:
with H5Flow(mode='w') as h5:
    grp1 = h5.create_group('couette', long_name='couette group')
    grp1.create_dataset_from_xarray_dataset(vel_couette)    
    
    grp2 = h5.create_group('poiseuille', long_name='poiseuille group')
    grp2.create_dataset_from_xarray_dataset(vel_poiseuille)
    hdf_filename = h5.hdf_filename
    
    u = h5.couette.u[:]
    h5.couette.mag[:].plot()
    print(h5.couette.u.__repr__())
    
    h5.dump()

A `xarray.Dataset` can be created from the hdf file to get the velocity components within one dataset:

In [ ]:
with H5Flow(hdf_filename, mode='r') as h5:
    h5.dump()
    vel = h5.couette.VelocityVector(standard_names=('x_velocity', 'y_velocity'))[:, :]

In [ ]:
vel

In [ ]:
vel.u.plot()

In [ ]:
u = vel.u[0,:]

In [ ]:
u.coords['y'].attrs.get('standard_name')